# Saving and Loading

Have questions? Chat with us on Github or Slack:

[![Homepage](https://img.shields.io/badge/fugue-source--code-red?logo=github)](https://github.com/fugue-project/fugue)
[![Slack Status](https://img.shields.io/badge/slack-join_chat-white.svg?logo=slack&style=social)](http://slack.fugue.ai)

So far, we've only covered modifying data with the `transform()` function. We constructed or loaded DataFrames with Pandas and then applied the transformation with a distributed computing engine. This setup will become a bottleneck for large files since we are loading everything at once on the driver node. On the other hand, loading a DataFrame using Spark, Dask, or Ray locks in the code to those frameworks.

In order to make end-to-end workflows that are compatible with all backends, Fugue exposes two main ways to to load and save data that are compatible with all backends. The first is with the `transform()` function. The second is using the `load()` and `save()` functions of the Fugue API.

## transform() using file path

The `transform()` function can take in a file path instead of a DataFrame to load in data before performing the transformation. The engine specified will be used to directly load the file. First, we make an example file:

In [10]:
import pandas as pd
from fugue import transform

df = pd.DataFrame({"a": [1,2]})
df.to_parquet("/tmp/f.parquet")

Now we use the Dask engine to load in the data and apply the `dummy()` function.

In [11]:
def dummy(df:pd.DataFrame) -> pd.DataFrame:
    return df

res = transform("/tmp/f.parquet", dummy, schema="*", engine="dask")
res.compute()

,a
0,1
1,2


To save the results, the `transform()` function can also take in a `save_path` argument. By default, it will return the path where it was saved, which is helpful for consecutive `transform()` calls.

In [12]:
transform("/tmp/f.parquet", dummy, schema="*", engine="dask", save_path="/tmp/f_out.parquet")

'/tmp/f_out.parquet'

## load() and save()

The Fugue API also has `load()` and `save()` methods that are compatible with any engine. These are capable of loading `parquet`, `csv`, and `json` files. Using `parquet` when possible is best practice because it contains schema information and does not require additional keywords to parse.

In [13]:
import fugue.api as fa

df = fa.load("/tmp/f.parquet", engine="dask")
res = fa.transform(df, dummy, schema="*", engine="dask")
fa.save(res, "/tmp/f_out.parquet", engine="dask")

## Summary

In this section, we learned how save and load DataFrames in an engine-agnostic way. Both methods presented in this section will work across all execution engines. In the last code snippet, we have to repeat `engine="dask"` three times. This can be redundant and tedious to type out. In practice, we can define the engine once by using the `engine_context()` we'll learn next section.